In [ ]:
import pickle
import jieba
import json

In [3]:
def init():
    f = open('./CAIL_data/law.txt', 'r', encoding = 'utf8')
    law = {}
    lawname = {}
    line = f.readline()
    while line:
        lawname[len(law)] = line.strip()
        law[line.strip()] = len(law)
        line = f.readline()
    f.close()


    f = open('./CAIL_data/accu.txt', 'r', encoding = 'utf8')
    accu = {}
    accuname = {}
    line = f.readline()
    while line:
        accuname[len(accu)] = line.strip()
        accu[line.strip()] = len(accu)
        line = f.readline()
    f.close()


    return law, accu, lawname, accuname


law, accu, lawname, accuname = init()

In [4]:
def getClassNum(kind):
    global law
    global accu

    if kind == 'law':
        return len(law)
    if kind == 'accu':
        return len(accu)

In [5]:
def getName(index, kind):
    global lawname
    global accuname
    if kind == 'law':
        return lawname[index]

    if kind == 'accu':
        return accuname[index]


def gettime(time):
    #将刑期用分类模型来做
    v = int(time['imprisonment'])

    if time['death_penalty']:
        return 0
    if time['life_imprisonment']:
        return 1
    elif v > 10 * 12:
        return 2
    elif v > 7 * 12:
        return 3
    elif v > 5 * 12:
        return 4
    elif v > 3 * 12:
        return 5
    elif v > 2 * 12:
        return 6
    elif v > 1 * 12:
        return 7
    else:
        return 8


def getlabel(d, kind):
    global law
    global accu    
    # 做单标签
    if kind == 'law':
    # 返回多个类的第一个
        return law[str(d['meta']['relevant_articles'][0])]
    if kind == 'accu':
        return accu[d['meta']['accusation'][0]]    
    if kind == 'time':
        return gettime(d['meta']['term_of_imprisonment'])
    
    return label
    


In [ ]:
def read_trainData(path):
    fin = open(path, 'r', encoding = 'utf8')
    
    alltext = []
    
    accu_label = []
    law_label = []
    time_label = []

    line = fin.readline()
    while line:
        d = json.loads(line)
        alltext.append(d['fact'])
        accu_label.append(getlabel(d, 'accu'))
        law_label.append(getlabel(d, 'law'))
        time_label.append(getlabel(d, 'time'))
        line = fin.readline()
    fin.close()

    return alltext, accu_label, law_label, time_label

In [ ]:
alltext, accu_label, law_label, time_label = read_trainData('./CAIL_data/data_train.json')

In [ ]:
def cut_text(alltext):

    train_text = []
    for text in alltext:
        train_text.append(' '.join(jieba.cut(text)))
     
    return train_text

train_data = cut_text(alltext)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer as TFIDF
from sklearn.externals import joblib

In [ ]:
def train_tfidf(train_data):
    tfidf = TFIDF(
            min_df = 5,
            max_features = 1000,
            ngram_range = (1, 3),
            use_idf = 1,
            smooth_idf = 1
            )
    tfidf.fit(train_data)
    
    return tfidf

In [ ]:
tfidf = train_tfidf(train_data)
vec = tfidf.transform(train_data)

joblib.dump(tfidf, 'predictor/model/tfidf.model')

In [ ]:
vec.shape, len(accu_label)

In [ ]:
########################################

In [26]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense,Dropout, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer as TFIDF
import json
#from predictor import data
from sklearn.svm import LinearSVC
from sklearn.externals import joblib
import pickle
#import thulac
import jieba
# from keras.layers import Dense, Dropout, Activation

In [ ]:
# Y = [1,3,4]

# def cate_label(Y):
#     # encode class values as integers
#     encoder = LabelEncoder()
#     encoder.fit(Y)
#     encoded_Y = encoder.transform(Y)
#     # convert integers to dummy variables (i.e. one hot encoded)
#     dummy_y = np_utils.to_categorical(encoded_Y)
#     return dummy_y


In [22]:
# define baseline model
def dnn_model(input_dim, output_dim):
    # create model
    model = Sequential()
    model.add(Dense(512, input_dim=1000, activation='relu'))
    #model.add(Dropout(0.3))
    #model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(output_dim, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [27]:
def train_DNN(vecs, label, label_index,batch_size = 2000):            
    #  模型学习
    # 构建一个CNN模型
    label = to_categorical(label, len(label_index))
    output_dim =label.shape[1] #output_dim = label.shape[1]
    input_dim  = vecs.shape[1]
    model = dnn_model(input_dim, output_dim)

    print('compile')
    model.fit(vecs, label, batch_size=2000, epochs=50, verbose=1, 
              callbacks=None, validation_split=0.0, validation_data=None, 
              shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0)
    
    
    print(model.summary())
    return model 

In [24]:
from keras.utils.np_utils import to_categorical

In [28]:
accu_model = train_DNN(vec,accu_label, accu)

compile
Epoch 1/50
154592/154592 [==============================] - 39s 253us/step - loss: 3.5293 - acc: 0.3491
Epoch 2/50
154592/154592 [==============================] - 38s 249us/step - loss: 1.3712 - acc: 0.6862
Epoch 3/50
154592/154592 [==============================] - 38s 243us/step - loss: 0.9463 - acc: 0.7665
Epoch 4/50
154592/154592 [==============================] - 38s 244us/step - loss: 0.7749 - acc: 0.8023
Epoch 5/50
154592/154592 [==============================] - 38s 245us/step - loss: 0.6706 - acc: 0.8253
Epoch 6/50
154592/154592 [==============================] - 38s 244us/step - loss: 0.5969 - acc: 0.8432
Epoch 7/50
154592/154592 [==============================] - 38s 244us/step - loss: 0.5396 - acc: 0.8554
Epoch 8/50
154592/154592 [==============================] - 38s 249us/step - loss: 0.4909 - acc: 0.8675
Epoch 9/50
154592/154592 [==============================] - 40s 256us/step - loss: 0.4488 - acc: 0.8779
Epoch 10/50
154592/154592 [=============================

In [29]:
law_model = train_DNN(vec, law_label, law)

compile
Epoch 1/50
154592/154592 [==============================] - 38s 249us/step - loss: 3.5014 - acc: 0.3375
Epoch 2/50
154592/154592 [==============================] - 38s 244us/step - loss: 1.4076 - acc: 0.6749
Epoch 3/50
154592/154592 [==============================] - 39s 254us/step - loss: 1.0057 - acc: 0.7485
Epoch 4/50
154592/154592 [==============================] - 38s 245us/step - loss: 0.8437 - acc: 0.7798
Epoch 5/50
154592/154592 [==============================] - 38s 245us/step - loss: 0.7457 - acc: 0.8005
Epoch 6/50
154592/154592 [==============================] - 38s 244us/step - loss: 0.6717 - acc: 0.8172
Epoch 7/50
154592/154592 [==============================] - 38s 245us/step - loss: 0.6143 - acc: 0.8300
Epoch 8/50
154592/154592 [==============================] - 38s 244us/step - loss: 0.5675 - acc: 0.8406
Epoch 9/50
154592/154592 [==============================] - 38s 245us/step - loss: 0.5249 - acc: 0.8510
Epoch 10/50
154592/154592 [=============================

In [30]:
time_model = train_DNN(vec, time_label, set(time_label))

compile
Epoch 1/50
154592/154592 [==============================] - 47s 302us/step - loss: 1.4176 - acc: 0.5127
Epoch 2/50
154592/154592 [==============================] - 37s 239us/step - loss: 1.1600 - acc: 0.5652
Epoch 3/50
154592/154592 [==============================] - 36s 235us/step - loss: 1.0849 - acc: 0.5909
Epoch 4/50
154592/154592 [==============================] - 36s 234us/step - loss: 1.0305 - acc: 0.6110
Epoch 5/50
154592/154592 [==============================] - 36s 234us/step - loss: 0.9734 - acc: 0.6348
Epoch 6/50
154592/154592 [==============================] - 36s 235us/step - loss: 0.9012 - acc: 0.6645
Epoch 7/50
154592/154592 [==============================] - 36s 235us/step - loss: 0.8103 - acc: 0.7044
Epoch 8/50
154592/154592 [==============================] - 36s 235us/step - loss: 0.7028 - acc: 0.7484
Epoch 9/50
154592/154592 [==============================] - 36s 235us/step - loss: 0.5840 - acc: 0.7964
Epoch 10/50
154592/154592 [=============================

In [31]:
from keras.models import model_from_json

In [32]:
#accu
# serialize model to JSON
accu_model_json = accu_model.to_json()
with open("./predictor/model/accu_model.json", "w") as json_file:
    json_file.write(accu_model_json)
# serialize weights to HDF5
accu_model.save_weights("./predictor/model/accu_model.h5")
print("Saved accu model to disk")
 
# later...
 
# load json and create model
json_file = open('./predictor/model/accu_model.json', 'r')
loaded_accu_model_json = json_file.read()
json_file.close()
accu_model = model_from_json(loaded_accu_model_json)
# load weights into new model
accu_model.load_weights("./predictor/model/accu_model.h5")
print("Loaded model from disk")

Saved accu model to disk
Loaded model from disk


In [33]:
#law
# serialize model to JSON
law_model_json = law_model.to_json()
with open("./predictor/model/law_model.json", "w") as json_file:
    json_file.write(law_model_json)
# serialize weights to HDF5
law_model.save_weights("./predictor/model/law_model.h5")
print("Saved law model to disk")
 
# later...
 
# load json and create model
json_file = open('./predictor/model/law_model.json', 'r')
loaded_law_model_json = json_file.read()
json_file.close()
law_model = model_from_json(loaded_law_model_json)
# load weights into new model
law_model.load_weights("./predictor/model/law_model.h5")
print("Loaded model from disk")

Saved law model to disk
Loaded model from disk


In [35]:
#time
# serialize model to JSON
time_model_json = time_model.to_json()
with open("./predictor/model/time_model.json", "w") as json_file:
    json_file.write(time_model_json)
# serialize weights to HDF5
time_model.save_weights("./predictor/model/time_model.h5")
print("Saved model to disk")
 
# later...
 
# load json and create model
json_file = open('./predictor/model/time_model.json', 'r')
loaded_time_model_json = json_file.read()
json_file.close()
time_model = model_from_json(loaded_time_model_json)
# load weights into new model
time_model.load_weights("./predictor/model/time_model.h5")
print("Loaded model from disk")


Saved model to disk
Loaded model from disk


In [ ]:
#####################################

In [ ]:
from keras.utils.np_utils import to_categorical
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D, GlobalMaxPooling1D


def getCNN(max_sequence_length, word_index, embedding_dim, embedding_matrix, output_dim, filters = [3,4,5]):
    
    print('call getCNN')
    print(len(word_index), output_dim)
    
    model = Sequential()

    model.add(Conv1D(128, 3, activation='relu'))
    model.add(MaxPooling1D(3))
    model.add(Conv1D(64, 3, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dropout(0.5))
    model.add(Dense(output_dim, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    return model




def train_CNN(vecs, label, label_index,batch_size = 2000):            
    #  模型学习
    # 构建一个CNN模型
    label = to_categorical(label, len(label_index))
    output_dim =label.shape[1] #output_dim = label.shape[1]
    model = getCNN(dataSet.max_sequence_length,
                   dataSet.word_index,
                   dataSet.embedding_dim,
                   dataSet.embedding_matrix, 
                   output_dim)

    print('compile')
    
    model.fit(vecs, label, batch_size=2000, epochs=10, verbose=1, 
              callbacks=None, validation_split=0.0, validation_data=None, 
              shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0)
    
    
    print(model.summary())
    return model 

In [ ]:
##################################################################################################################

In [1]:
import json
import numpy as np
from sklearn.externals import joblib
from keras.models import model_from_json

# 斯坦福分词，选取有意义的名词，专有名词，和动词
# from stanfordcorenlp import StanfordCoreNLP
# nlp = StanfordCoreNLP('/home/jeshe/stanford-corenlp-full-2017-06-09/', lang='zh')

# def seg_document(document):               
#     result = [i for (i, j) in nlp.pos_tag(document) if j in ['NN', 'NR', 'VV']]
#     return result

import jieba
class Predictor(object):
    def __init__(self):
        self.tfidf = joblib.load('predictor/model/tfidf.model')
        
        # load json and create model
        json_file = open('./predictor/model/accu_model.json', 'r')
        loaded_accu_model_json = json_file.read()
        json_file.close()
        self.accu_model = model_from_json(loaded_accu_model_json)
        # load weights into new model
        self.accu_model.load_weights("./predictor/model/accu_model.h5")
        print("Loaded accu model from disk")
        
        json_file = open('./predictor/model/law_model.json', 'r')
        loaded_law_model_json = json_file.read()
        json_file.close()
        self.law_model = model_from_json(loaded_law_model_json)
        # load weights into new model
        self.law_model.load_weights("./predictor/model/law_model.h5")
        print("Loaded law model from disk")

        json_file = open('./predictor/model/time_model.json', 'r')
        loaded_time_model_json = json_file.read()
        json_file.close()
        self.time_model = model_from_json(loaded_time_model_json)
        # load weights into new model
        self.time_model.load_weights("./predictor/model/time_model.h5")
        print("Loaded time model from disk")

        
        
#         self.law = joblib.load('predictor/model/law.model')
#         self.accu = joblib.load('predictor/model/accu.model')
#         self.time = joblib.load('predictor/model/time.model')
        self.batch_size = 1
        
#         self.accumodel = joblib.load('predictor/model/accu.model')
        
        #self.cut = thulac.thulac(seg_only = True)
        


    def predict_law(self, vec):
        y = self.law_model.predict(vec)
        y = np.argmax(y, axis=1)
#         return y
        return [y[0] + 1]
    
    def predict_accu(self, vec):
        y = self.accu_model.predict(vec)
        y = np.argmax(y, axis=1)
#         return y
        return [y[0] + 1]
    
    def predict_time(self, vec):

        y = self.time_model.predict(vec)[0]
        y = np.argmax(y)
        
        #返回每一个罪名区间的中位数
        if y == 0:
            return -2
        if y == 1:
            return -1
        if y == 2:
            return 120
        if y == 3:
            return 102
        if y == 4:
            return 72
        if y == 5:
            return 48
        if y == 6:
            return 30
        if y == 7:
            return 18
        else:
            return 6
    
    
    def predict(self, content):
        fact = []
        #print('text cut...')
        #print(content)
        for text in content:
            fact.append(' '.join(jieba.cut(text)))
        #print(fact)
#         fact = self.cut.cut(content[0], text = True)
#         print(fact[0])
        vec = self.tfidf.transform(fact)
        ans = {}

        ans['accusation'] = self.predict_accu(vec)
        ans['articles'] = self.predict_law(vec)
        ans['imprisonment'] = self.predict_time(vec)
        
        #print(ans)
        return [ans]


/home/dtt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import json
import os
import multiprocessing

#from predictor import Predictor

data_path = "./input/"  # The directory of the input data
output_path = "./output/"  # The directory of the output data


def format_result(result):
    rex = {"accusation": [], "articles": [], "imprisonment": -3}

    res_acc = []
    for x in result["accusation"]:
        if not (x is None):
            res_acc.append(int(x))
    rex["accusation"] = res_acc

    if not (result["imprisonment"] is None):
        rex["imprisonment"] = int(result["imprisonment"])
    else:
        rex["imprisonment"] = -3

    res_art = []
    for x in result["articles"]:
        if not (x is None):
            res_art.append(int(x))
    rex["articles"] = res_art

    return rex


if __name__ == "__main__":
    user = Predictor()
    cnt = 0


    def get_batch():
        v = user.batch_size
        if not (type(v) is int) or v <= 0:
            raise NotImplementedError

        return v


    def solve(fact):
        result = user.predict(fact)

        for a in range(0, len(result)):
            result[a] = format_result(result[a])

        return result


    for file_name in os.listdir(data_path):
        inf = open(os.path.join(data_path, file_name), "r")
        ouf = open(os.path.join(output_path, file_name), "w")

        fact = []

        for line in inf:
            fact.append(json.loads(line)["fact"])
            if len(fact) == get_batch():
                result = solve(fact)
                cnt += len(result)
                for x in result:
                    print(json.dumps(x), file=ouf)
                fact = []

        if len(fact) != 0:
            result = solve(fact)
            cnt += len(result)
            for x in result:
                print(json.dumps(x), file=ouf)
            fact = []

        ouf.close()


Loaded accu model from disk
Loaded law model from disk


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


Loaded time model from disk


Loading model cost 1.288 seconds.
Prefix dict has been built succesfully.


In [ ]:
###########################################

In [ ]:
from math import log
import os
import json


class Judger:
    # Initialize Judger, with the path of accusation list and law articles list
    def __init__(self, accusation_path, law_path):
        self.accu_dic = {}

        f = open(accusation_path, "r")
        self.task1_cnt = 0
        for line in f:
            self.task1_cnt += 1
            self.accu_dic[line[:-1]] = self.task1_cnt

        self.law_dic = {}
        f = open(law_path, "r")
        self.task2_cnt = 0
        for line in f:
            self.task2_cnt += 1
            self.law_dic[int(line[:-1])] = self.task2_cnt

    # Format the result generated by the Predictor class
    @staticmethod
    def format_result(result):
        rex = {"accusation": [], "articles": [], "imprisonment": -3}

        res_acc = []
        for x in result["accusation"]:
            if not (x is None):
                res_acc.append(int(x))
        rex["accusation"] = res_acc

        if not (result["imprisonment"] is None):
            rex["imprisonment"] = int(result["imprisonment"])
        else:
            rex["imprisonment"] = -3

        res_art = []
        for x in result["articles"]:
            if not (x is None):
                res_art.append(int(x))
        rex["articles"] = res_art

        return rex

    # Gen new results according to the truth and users output
    def gen_new_result(self, result, truth, label):
        s1 = set(label["accusation"])
        s2 = set()
        for name in truth["accusation"]:
            s2.add(self.accu_dic[name.replace("[", "").replace("]", "")])

        for a in range(0, self.task1_cnt):
            in1 = (a + 1) in s1
            in2 = (a + 1) in s2
            if in1:
                if in2:
                    result[0][a]["TP"] += 1
                else:
                    result[0][a]["FP"] += 1
            else:
                if in2:
                    result[0][a]["FN"] += 1
                else:
                    result[0][a]["TN"] += 1

        s1 = set(label["articles"])
        s2 = set()
        for name in truth["relevant_articles"]:
            s2.add(self.law_dic[name])

        for a in range(0, self.task2_cnt):
            in1 = (a + 1) in s1
            in2 = (a + 1) in s2
            if in1:
                if in2:
                    result[1][a]["TP"] += 1
                else:
                    result[1][a]["FP"] += 1
            else:
                if in2:
                    result[1][a]["FN"] += 1
                else:
                    result[1][a]["TN"] += 1

        result[2]["cnt"] += 1
        sc = 0
        if truth["term_of_imprisonment"]["death_penalty"]:
            if label["imprisonment"] == -2:
                sc = 1
        elif truth["term_of_imprisonment"]["life_imprisonment"]:
            if label["imprisonment"] == -1:
                sc = 1
        else:
            if label["imprisonment"] < 0:
                sc = 0
            else:
                v1 = truth["term_of_imprisonment"]["imprisonment"]
                v2 = label["imprisonment"]
                v = abs(log(v1 + 1) - log(v2 + 1))
                if v <= 0.2:
                    sc = 1
                elif v <= 0.4:
                    sc = 0.8
                elif v <= 0.6:
                    sc = 0.6
                elif v <= 0.8:
                    sc = 0.4
                elif v <= 1.0:
                    sc = 0.2
                else:
                    sc = 0
        sc = sc * 1.0
        result[2]["score"] += sc

        return result

    # Calculate precision, recall and f1 value
    # According to https://github.com/dice-group/gerbil/wiki/Precision,-Recall-and-F1-measure
    @staticmethod
    def get_value(res):
        if res["TP"] == 0:
            if res["FP"] == 0 and res["FN"] == 0:
                precision = 1.0
                recall = 1.0
                f1 = 1.0
            else:
                precision = 0.0
                recall = 0.0
                f1 = 0.0
        else:
            precision = 1.0 * res["TP"] / (res["TP"] + res["FP"])
            recall = 1.0 * res["TP"] / (res["TP"] + res["FN"])
            f1 = 2 * precision * recall / (precision + recall)

        return precision, recall, f1

    # Generate score for the first two subtasks
    def gen_score(self, arr):
        sumf = 0
        y = {"TP": 0, "FP": 0, "FN": 0, "TN": 0}
        for x in arr:
            p, r, f = self.get_value(x)
            sumf += f
            for z in x.keys():
                y[z] += x[z]

        _, __, f_ = self.get_value(y)

        return (f_ + sumf * 1.0 / len(arr)) / 2.0

    # Generatue all scores
    def get_score(self, result):
        s1 = self.gen_score(result[0])
        s2 = self.gen_score(result[1])
        s3 = 1.0 * result[2]["score"] / result[2]["cnt"]
        return [s1, s2, s3]

    # Test with ground truth path and the user's output path
    def test(self, truth_path, output_path):
        cnt = 0
        result = [[], [], {}]
        for a in range(0, self.task1_cnt):
            result[0].append({"TP": 0, "FP": 0, "TN": 0, "FN": 0})
        for a in range(0, self.task2_cnt):
            result[1].append({"TP": 0, "FP": 0, "TN": 0, "FN": 0})
        result[2] = {"cnt": 0, "score": 0}

        for file_name in os.listdir(truth_path):
            inf = open(os.path.join(truth_path, file_name), "r")
            ouf = open(os.path.join(output_path, file_name), "r")

            for line in inf:
                ground_truth = json.loads(line)["meta"]
                user_output = json.loads(ouf.readline())

                cnt += 1
                result = self.gen_new_result(result, ground_truth, user_output)

        return result
